### Loading Docx

In [ ]:
from docx import Document

In [ ]:
import  os
Input_dir = './Inputs/'
os.listdir(Input_dir)

['.ipynb_checkpoints',
 'demo.docx',
 'input-plain-document (1).txt',
 'mydoc.doc',
 'mydoc.docx']

In [ ]:
docs = []
for file in os.listdir(Input_dir):
    if file.endswith('.docx'):
        path = Input_dir +  file
        docs.append((path, Document(path)))  

In = []
for path, doc in docs:
    In.append((path, [para.text for para in doc.paragraphs]))

In [ ]:
doc.paragraphs[4].style.font.size

177800

In [ ]:
doc.paragraphs[4].text

'Name of the medicine'

In [ ]:
In[0][1] # Read like this In[nth document][path=0, full text =1][nth text]

['Demonstration of DOCX support in calibre',
 'This document demonstrates the ability of the calibre DOCX Input plugin to convert the various typographic features in a Microsoft Word (2007 and newer) document. Convert this document to a modern ebook format, such as AZW3 for Kindles or EPUB for other ebook readers, to see it in action.',
 'There is support for images, tables, lists, footnotes, endnotes, links, dropcaps and various types of text and paragraph level formatting.',
 'To see the DOCX conversion in action, simply add this file to calibre using the “Add Books” button and then click “Convert”.  Set the output format in the top right corner of the conversion dialog to EPUB or AZW3 and click “OK”.',
 '',
 'Text Formatting',
 'Inline formatting',
 'Here, we demonstrate various types of inline text formatting and the use of embedded fonts.',
 'Here is some bold, italic, bold-italic, underlined and struck out  text. Then, we have a superscript and a subscript. Now we see some red, g

### Loading .doc 

In [ ]:
#!pip install pywin32

In [ ]:
#!pip install textract

In [ ]:
from win32com import client

In [ ]:
# Current Working Directory
cwd = os.getcwd()

In [ ]:
docs = []
for file in os.listdir(Input_dir):
    path = Input_dir +  file
    # Converting doc to docx
    if path.endswith('.doc'):
        word = client.Dispatch("word.application")  # Opening MS Word
        doc = word.Documents.Open(cwd + path[1:]) # Opening the doc
        doc.SaveAs(cwd + '/temp/temp.docx',FileFormat = client.constants.wdFormatXMLDocument) # Saving as docx using word.
        doc.Close()
        word.Quit()
        # Reading the temp docx
        doc = Document('./temp/temp.docx')
        In.append((path, [para.text for para in doc.paragraphs]))
        

### Loading txt:

In [ ]:
import chardet
for file in os.listdir(Input_dir):
    if file.endswith('.txt'):
        path = Input_dir +  file
        with open(path,'r') as f: # Assuming standard encodings are used
            texts = f.read().splitlines()
        In.append((path,texts))

### Predicting

In [ ]:
# Loading PreProcesser
from dill import load
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
with open('./Data/Models/preprocess.pkl','rb') as f:
    preprocess = load(f)
# Loading Model
from keras.models import load_model
model = load_model('./Data/Models/Model.h5')

In [ ]:
def decode(y):
    return np.argmax(y,1)

In [ ]:
# Insert Code to Load model as model
    
y = []

for path, texts in In:
    t= []
#     for text in texts:
#         t.append(model.predict(preprocess.transform(text)))
    t = model.predict(preprocess.transform(texts))
    y.append(decode(t))

### Output

In [ ]:
# Assign tags
h = 0  #heading
sh = 1 #sub-heading
s = 2 #section

#### Texts

In [ ]:
for i, tags in enumerate(y):
    path = In[i][0]
    doc = In[i][1]
    if path.endswith('.txt'):
        flag = 0 # Flag = 1 if start section is open and not closed yet
        with open(path.replace('./Inputs/','./Outputs/'), 'w') as f:
            for i, text in enumerate(doc):
                if tags[i]==h:
                    f.write('<<START_HEADING>>' + text + '<<END_HEADING>>\n' )
                if tags[i]==sh:
                    f.write('<<START_SUBHEADING>>' + text + '<<END_SUBHEADING>>\n' )
                if tags[i]==s:
                    if flag==0: # Start section is not open
                        if i==len(tags)-1: # Checking if this is the last section
                            f.write('<<START_SECTION>>' + text + '<<END_SECTION>>\n')
                            break
                        if tags[i+1]==s: # Checking if next tag is section
                            f.write('<<START_SECTION>>' + text + '\n')
                            flag=1 # Section tag started
                        else:
                            f.write('<<START_SECTION>>' + text + '<<END_SECTION>>\n')
                    if flag==1:
                        if i==len(tags)-1: # Checking if this is the last section
                            f.write(text + '<<END_SECTION>>\n')
                            break
                        if tags[i+1]!=s:
                            f.write(text + '<<END_SECTION>>\n')
                            flag=0 # Section tag end
                        else:
                            f.write(text+'\n')

#### Docx

In [ ]:
for i, tags in enumerate(y):
    path = In[i][0]
    if path.endswith('.docx'):
        doc = Document(path)
        flag=0
        for i, para in enumerate(doc.paragraphs):
                text = para.text
                if tags[i]==h:
                    para.text = para.text.replace(text,'<<START_HEADING>>' + text + '<<END_HEADING>>\n')

                if tags[i]==sh:
                    para.text = para.text.replace(text,'<<START_SUBHEADING>>' + text + '<<END_SUBHEADING>>\n')

                if tags[i]==s:
                    if flag==0:
                        if i==len(tags)-1:
                            para.text = para.text.replace(text,'<<START_SECTION>>' + text + '<<END_SECTION>>\n')
                            break
                        if tags[i+1]==s:
                            para.text = para.text.replace(text,'<<START_SECTION>>' + text + '\n')
                            flag=1
                        else:
                            para.text = para.text.replace(text,'<<START_SECTION>>' + text + '<<END_SECTION>>\n')
                    if flag==1:
                        if i==len(tags)-1:
                            para.text = para.text.replace(text, text + '<<END_SECTION>>\n')
                            break
                            
                        if tags[i+1]!=s:
                            para.text = para.text.replace(text, text + '<<END_SECTION>>\n')
                            flag=0
                        else:
                            para.text = para.text.replace(text,text + '\n')
                    
        doc.save(path.replace('./Inputs/','./Outputs/'))

### Doc

In [ ]:
for i, tags in enumerate(y):
    path = In[i][0]
    # Converting doc to docx
    if path.endswith('.doc'):
        word = client.Dispatch("word.application")
        doc = word.Documents.Open(cwd + path[1:])
        doc.SaveAs(cwd + '/temp/temp.docx',FileFormat = client.constants.wdFormatXMLDocument )
        doc.Close()
        word.Quit()
    # Editing docx and storing temporarily 
        doc = Document('./temp/temp.docx')
        for i, para in enumerate(doc.paragraphs):
                text = para.text
                if tags[i]==h:
                    para.text = para.text.replace(text,'<<START_HEADING>>' + text + '<<END_HEADING>>\n')

                if tags[i]==sh:
                    para.text = para.text.replace(text,'<<START_SUBHEADING>>' + text + '<<END_SUBHEADING>>\n')

                if tags[i]==s:
                    if flag==0:
                        if i==len(tags)-1:
                            para.text = para.text.replace(text,'<<START_SECTION>>' + text + '<<END_SECTION>>\n')
                            break
                        if tags[i+1]==s:
                            para.text = para.text.replace(text,'<<START_SECTION>>' + text + '\n')
                            flag=1
                        else:
                            para.text = para.text.replace(text,'<<START_SECTION>>' + text + '<<END_SECTION>>\n')
                    if flag==1:
                        if i==len(tags)-1:
                            para.text = para.text.replace(text, text + '<<END_SECTION>>\n')
                            break
                        if tags[i+1]!=s:
                            para.text = para.text.replace(text, text + '<<END_SECTION>>\n')
                            flag=0
                        else:
                            para.text = para.text.replace(text,text + '\n')
        doc.save(cwd + '/temp/temp.docx')
        
    # Converting Back to doc 
        word = client.Dispatch("word.application")
        doc = word.Documents.Open(cwd + './temp/temp.docx')
        doc.SaveAs(cwd + path.replace('./Inputs/','./Outputs/'))
        doc.Close()
        word.Quit()